<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.3:
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and database maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands".

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [4]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

#the code gives error because there is no table named houses

OperationalError: no such table: houses

In [5]:
c.execute('''
CREATE TABLE houses(
    field1 INTEGER PRIMARY KEY, 
    sqft INTEGER, 
    bdrms INTEGER, 
    age INTEGER,
    price INTEGER);''')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [6]:
# Adding data
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key.

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [7]:
# Adding many data
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
#
c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

Select all rows from houses

In [8]:
# Selecting all rows from house
query = 'Select * from houses'
c.execute(query)
c.fetchall()
#pd.DataFrame(c.fetchall())

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more.

In [9]:
#adding data from the excel file
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
path = r"C:\Users\Vinitha\Desktop\IOD\DATA\housing-data.csv"

data = genfromtxt(path, dtype='i8', delimiter=',', skip_header=1).tolist()
print(data[:5])

[[2104, 3, 70, 399900], [1600, 3, 28, 329900], [2400, 3, 44, 369000], [1416, 2, 49, 232000], [3000, 4, 75, 539900]]


Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [10]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

Now we can insert each list item as a row of fields in the database.

In [11]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In [12]:
#printing the type and first 3 rows
print(type(data))
print(data[:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [13]:
#inserting one new row with none as the first row value
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [14]:
# deleting rows where the field1 index is 52,53
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?
- The command would delete all rows from the table named houses in the database where this query is executed.

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [15]:
# ANSWER
# querying only housies where the bedrooms is 4
query = "SELECT * FROM houses WHERE bdrms = 4"
c.execute(query)
c.fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. The following query calculates the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [16]:
# querying to view the average sqft and average price based on the number of bedrooms
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [17]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database.

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [18]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv(path, low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [19]:
# storing Housing Data from Pandas to SQLite
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

47

Run a query to get the average price of each house size from this table:

In [20]:
# querying to view the sqft and average price based on the number of bedrooms

result = c.execute("SELECT sqft, AVG(price) FROM houses GROUP BY sqft")
result.fetchall()

[(852, 179900.0),
 (1000, 169900.0),
 (1100, 249900.0),
 (1200, 299000.0),
 (1203, 239500.0),
 (1236, 199900.0),
 (1239, 229900.0),
 (1268, 259900.0),
 (1320, 299900.0),
 (1380, 212000.0),
 (1416, 232000.0),
 (1427, 198999.0),
 (1437, 249900.0),
 (1458, 464500.0),
 (1494, 242500.0),
 (1505, 269000.0),
 (1534, 314900.0),
 (1600, 329900.0),
 (1604, 242900.0),
 (1664, 368500.0),
 (1767, 252900.0),
 (1811, 285900.0),
 (1839, 349900.0),
 (1852, 299900.0),
 (1870, 289000.0),
 (1888, 255000.0),
 (1890, 329999.0),
 (1940, 239999.0),
 (1962, 259900.0),
 (1985, 299900.0),
 (2000, 347000.0),
 (2040, 314900.0),
 (2104, 399900.0),
 (2132, 345000.0),
 (2162, 287000.0),
 (2200, 475000.0),
 (2238, 329900.0),
 (2300, 449900.0),
 (2390, 319000.0),
 (2400, 369000.0),
 (2526, 469000.0),
 (2567, 314000.0),
 (2609, 499998.0),
 (2637, 299900.0),
 (3000, 539900.0),
 (3031, 599000.0),
 (3137, 579900.0),
 (3890, 573900.0),
 (4000, 619000.0),
 (4215, 549000.0),
 (4478, 699900.0)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?
   - I would use Pandas to clean, preprocess, and structure data before inserting it into SQLite

3. When would you want to use Pandas on a dataset retrieved from a database?
   - would use pandas to analyze, visualize, and further manipulate data retrieved from SQLite

<a name="guided-practice"></a>


# Reference: SQL Syntax


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate functions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [21]:
# Reading CSV to Dataframe
orders = pd.read_csv('C:/Users/Vinitha/Desktop/IOD/DATA/P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('C:/Users/Vinitha/Desktop/IOD/DATA/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [22]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [23]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [24]:
# ANSWER
#replaceing spaces with _ and column names to lowercases
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [27]:
# ANSWER
#replaceing spaces with _ and column names to lowercases
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes

In [28]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [29]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [32]:
# Establishing Local DB connection
db_connection = sqlite3.connect('C:/Users/Vinitha/Desktop/IOD/DATA/eshop.db.sqlite')

# Save these two dataframes as a table in sqlite
orders.to_sql('orders', conn , if_exists='replace', index=False)
orders_break_down.to_sql('orders_break_down', conn , if_exists='replace', index=False)

8047

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [33]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT *
        FROM orders
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [34]:
# Select Number of Distinct Orders for Each Customer
query = """
        SELECT customer_name, COUNT(DISTINCT order_id) AS num_orders
        FROM orders
        GROUP BY customer_name;
        """
orders_distinct = sql.read_sql(query, con=db_connection)
orders_distinct

,customer_name,num_orders
0,Aaron Bergman,11
1,Aaron Hawkins,7
2,Aaron Smayling,12
3,Adam Bellavance,4
4,Adam Hart,8
...,...,...
787,Xylona Preis,2
788,Yana Sorensen,5
789,Yoseph Carroll,11
790,Zuschuss Carroll,9


#### 6. Select Number of Customers for Each Country

In [35]:
# Select Number of Customers for Each Country
query = """
        SELECT country, COUNT(DISTINCT customer_name) AS number_of_customers
        FROM orders
        GROUP BY country;
        """
number_of_customers = sql.read_sql(query, con=db_connection)
number_of_customers

,country,number_of_customers
0,Austria,125
1,Belgium,66
2,Denmark,28
3,Finland,33
4,France,568
5,Germany,505
6,Ireland,48
7,Italy,351
8,Netherlands,172
9,Norway,37


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [36]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
query = """
        SELECT order_id,discount,sales,quantity	
        FROM orders_break_down;
        """
discount_sales_quantity_each_order = sql.read_sql(query, con=db_connection)
discount_sales_quantity_each_order

,order_id,discount,sales,quantity
0,IT-2011-3647632,0.5,45.0,3
1,ES-2011-4869686,0.0,854.0,7
2,ES-2011-4939443,0.0,140.0,3
3,IT-2011-2942451,0.5,27.0,2
4,IT-2011-2942451,0.5,17.0,2
...,...,...,...,...
8042,ES-2014-3458802,0.0,245.0,2
8043,IT-2014-3715679,0.5,30.0,2
8044,IT-2014-3715679,0.5,23.0,4
8045,IT-2014-3715679,0.5,108.0,3


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [37]:
# Select discount, sales, quantity, total price for Each Order Id from orders_break_down Table
query = """
        SELECT order_id ,discount ,sales ,quantity, SUM(sales * quantity * (1 - discount / 100)) AS total_price_after_discount
        FROM orders_break_down
        GROUP BY order_id;
        """

total_price_after_discount = sql.read_sql(query, con=db_connection)
total_price_after_discount

,order_id,discount,sales,quantity,total_price_after_discount
0,ES-2011-1001989,0.1,229.0,9,2058.939
1,ES-2011-1010958,0.0,58.0,2,956.000
2,ES-2011-1012469,0.0,148.0,3,444.000
3,ES-2011-1043483,0.0,224.0,11,6562.000
4,ES-2011-1058269,0.0,13.0,2,11139.008
...,...,...,...,...,...
4112,IT-2014-5966070,0.5,107.0,4,425.860
4113,IT-2014-5975833,0.1,656.0,2,3804.192
4114,IT-2014-5984498,0.5,83.0,1,82.585
4115,IT-2014-5989338,0.1,1603.0,9,14412.573


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [38]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
query = '''
SELECT *
FROM orders_break_down
WHERE (sales * quantity * (1 - discount / 100)) > 100;
        '''
total_price_greater_than_100 = sql.read_sql(query, con=db_connection)
total_price_greater_than_100

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,ES-2011-3848439,"Binney & Smith Pencil Sharpener, Water Color",0.0,90.0,3,Office Supplies
4,ES-2011-3848439,"Sanford Canvas, Fluorescent",0.0,207.0,4,Office Supplies
...,...,...,...,...,...,...
6461,ES-2014-2815584,"Kleencut Shears, Serrated",0.0,261.0,6,Office Supplies
6462,ES-2014-4785777,"Wilson Jones Index Tab, Economy",0.0,32.0,5,Office Supplies
6463,ES-2014-3458802,"Ikea Stackable Bookrack, Traditional",0.0,245.0,2,Furniture
6464,IT-2014-3715679,"StarTech Phone, Red",0.5,108.0,3,Technology


#### 8. Select All Customers And The Products They Have Bought

In [50]:
# Select All Order Ids, Customers And The Products (product_name) They Have Bought
query = '''
SELECT orders_break_down.order_id, orders_break_down.product_name, orders.customer_name
FROM orders_break_down
JOIN orders ON orders_break_down.order_id = orders.order_id;
        '''
all_customers_and_their_products  = sql.read_sql(query, con=db_connection)
all_customers_and_their_products

,order_id,product_name,customer_name
0,IT-2011-3647632,"Enermax Note Cards, Premium",Eugene Moren
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",Dorothy Dickinson
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",Arthur Prichep
3,IT-2011-2942451,"Boston Markers, Easy-Erase",Grant Thornton
4,IT-2011-2942451,"Eldon Folders, Single Width",Grant Thornton
...,...,...,...
8042,ES-2014-3458802,"Ikea Stackable Bookrack, Traditional",John Grady
8043,IT-2014-3715679,"BIC Pencil Sharpener, Fluorescent",Jim Kriz
8044,IT-2014-3715679,"Avery Binder Covers, Recycled",Jim Kriz
8045,IT-2014-3715679,"StarTech Phone, Red",Jim Kriz


#### 9.A Select Number of 'Furniture' Orders For Each Country

In [63]:
# Select Number of 'Furniture' Orders For Each Country
query = '''
SELECT orders.country, COUNT(category) AS furniture_orders_count
FROM orders_break_down
JOIN orders ON orders_break_down.order_id = orders.order_id
WHERE orders_break_down.category = 'Furniture'
GROUP BY orders.country;
'''

furniture_order_for_each_country  = sql.read_sql(query, con=db_connection)
furniture_order_for_each_country

,country,furniture_orders_count
0,Austria,40
1,Belgium,20
2,Denmark,9
3,Finland,16
4,France,299
5,Germany,264
6,Ireland,12
7,Italy,136
8,Netherlands,76
9,Norway,6


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [67]:
# Select Number of 'Furniture' Orders For The Country Denmark
query = '''
SELECT orders.country, COUNT(category) AS furniture_orders_count
FROM orders_break_down
JOIN orders ON orders_break_down.order_id = orders.order_id
WHERE orders_break_down.category = 'Furniture' AND orders.country = 'Denmark'
GROUP BY orders.country;
'''

furniture_order_for_denmark  = sql.read_sql(query, con=db_connection)
furniture_order_for_denmark

,country,furniture_orders_count
0,Denmark,9


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [76]:
# Select Total Sales With Discount (discount > 0) and Without Discount (discount = 0) for Each Country
query = '''
SELECT orders.country,
       SUM(CASE WHEN orders_break_down.discount = 0 THEN orders_break_down.sales * orders_break_down.quantity ELSE 0 END) AS total_price_without_discount,
       SUM(CASE WHEN orders_break_down.discount > 0 THEN orders_break_down.sales * orders_break_down.quantity * (1 - orders_break_down.discount / 100) ELSE 0 END) AS total_price_with_discount
FROM orders_break_down 
JOIN orders ON orders_break_down.order_id = orders.order_id
GROUP BY orders.country;
'''

total_price_without_discount  = sql.read_sql(query, con=db_connection)
total_price_without_discount

,country,total_price_without_discount,total_price_with_discount
0,Austria,391694.0,0.000
1,Belgium,229829.0,0.000
2,Denmark,0.0,34442.008
3,Finland,82001.0,0.000
4,France,665914.0,2438890.490
5,Germany,1206817.0,1202885.758
6,Ireland,0.0,75344.596
7,Italy,806777.0,369575.090
8,Netherlands,0.0,377874.369
9,Norway,99878.0,0.000


#### 11.A Select Total Quantity, Total Sales for Each Country

In [87]:
# Select Total Quantity, Total Sales for Each Country
query = '''
SELECT 
SUM(orders_break_down.sales * orders_break_down.quantity) AS total_sales ,
SUM(orders_break_down.quantity) AS total_quantity, 
orders.country 
FROM orders_break_down
JOIN orders orders ON orders_break_down.order_id = orders.order_id
GROUP BY orders.country;
'''

quantity_sales_country = sql.read_sql(query, con=db_connection)
quantity_sales_country

,total_sales,total_quantity,country
0,391694.0,973,Austria
1,229829.0,532,Belgium
2,34617.0,204,Denmark
3,82001.0,201,Finland
4,3108592.0,7329,France
5,2411483.0,6179,Germany
6,75724.0,392,Ireland
7,1177921.0,3612,Italy
8,379714.0,1526,Netherlands
9,99878.0,261,Norway


#### 11.B Select Top 3 Countries and Quantities Based on Total Sales

In [95]:
# Select Top 3 Countries and Quantities Based on Total Sales
query = '''
SELECT orders.country, SUM(orders_break_down.sales * orders_break_down.quantity) AS total_quantity
FROM orders_break_down 
JOIN orders  ON orders_break_down.order_id = orders.order_id
GROUP BY orders.country
ORDER BY total_quantity DESC
LIMIT 3;

'''

top_three_countries = sql.read_sql(query, con=db_connection)
top_three_countries

,country,total_quantity
0,France,3108592.0
1,Germany,2411483.0
2,United Kingdom,2161947.0


#### 11.C Select Bottom 3 Countries and Sales Based On Total Quantities

In [96]:
# Select Bottom 3 Countries and Sales Based On Total Quantities
query = '''
SELECT orders.country, SUM(orders_break_down.quantity) AS total_quantity
FROM orders_break_down 
JOIN orders  ON orders_break_down.order_id = orders.order_id
GROUP BY orders.country
ORDER BY total_quantity ASC
LIMIT 3;

'''

bottom_three_countries = sql.read_sql(query, con=db_connection)
bottom_three_countries

,country,total_quantity
0,Finland,201
1,Denmark,204
2,Norway,261


#### 12. Select Average Sales By Category For The Country 'France'

In [98]:
# Select Average Sales By Category For The Country 'France'
query = '''
SELECT orders_break_down.category, AVG(orders_break_down.sales * orders_break_down.quantity * (1 - orders_break_down.discount / 100)) AS average_sales
FROM orders_break_down 
JOIN orders  ON orders_break_down.order_id = orders.order_id
WHERE orders.country = 'France'
GROUP BY orders_break_down.category;

'''

average_sales = sql.read_sql(query, con=db_connection)
average_sales

,category,average_sales
0,Furniture,3087.198483
1,Office Supplies,846.657519
2,Technology,2952.064087


#### 13. Select Country, Category and Average Sales Where Average Sales is The Highest

In [99]:
# Select Country, Category and Average Sales Where Average Sales is The Highest
query = '''
SELECT country, category, average_sales
FROM (SELECT orders.country, orders_break_down.category, AVG(orders_break_down.sales * orders_break_down.quantity * (1 - orders_break_down.discount / 100)) AS average_sales,
RANK() OVER (ORDER BY AVG(orders_break_down.sales * orders_break_down.quantity * (1 - orders_break_down.discount / 100)) DESC) AS rank_order
FROM orders_break_down 
JOIN orders  ON orders_break_down.order_id = orders.order_id
GROUP BY orders.country, orders_break_down.category) AS ranked_sales
WHERE rank_order = 1;
'''

average_sales = sql.read_sql(query, con=db_connection)
average_sales

,country,category,average_sales
0,Switzerland,Technology,6487.846154


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams.
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  



---



---



> > > > > > > > > © 2023 Institute of Data


---



---



